In [11]:
from lomond import WebSocket
import ccxt
import cryptopia
import os
import time
import talib
import logging
import requests
import datetime
import importlib
import dateutil.parser
import numpy as np
import pandas as pd
from importlib import reload
from collections import defaultdict
import sys
import inspect
import pyti
import math
data_directory = 'D:\\crypto_analysis-master-master\\crypto_analysis-master-master\\data\\'

In [12]:
df_csv_1day = pd.read_csv(data_directory+'BTC_Bitfinex_day_full_1day.csv', index_col=None,dayfirst=True)
df_csv_1day['timeframe'] = "1 day"
df_csv_3days = pd.read_csv(data_directory+'BTC_Bitfinex_day_full_3days.csv', index_col=None,dayfirst=True)
df_csv_3days['timeframe'] = "3 days"
df_csv_7days = pd.read_csv(data_directory+'BTC_Bitfinex_day_full_7days.csv', index_col=None,dayfirst=True)
df_csv_7days['timeframe'] = "7 days"
df_csv_1hour = pd.read_csv(data_directory+'BTC_Bitfinex_hour_full_1hour.csv', index_col=None,dayfirst=True)
df_csv_1hour['timeframe'] = "1 hour"
df_csv_4hours = pd.read_csv(data_directory+'BTC_Bitfinex_hour_full_4hours.csv', index_col=None,dayfirst=True)
df_csv_4hours['timeframe'] = "4 hours"
df_csv_6hours = pd.read_csv(data_directory+'BTC_Bitfinex_hour_full_6hours.csv', index_col=None,dayfirst=True)
df_csv_6hours['timeframe'] = "6 hours"
df_csv_12hours = pd.read_csv(data_directory+'BTC_Bitfinex_hour_full_12hours.csv', index_col=None,dayfirst=True)
df_csv_12hours['timeframe'] = "12 hours"
df_csv_14days = pd.read_csv(data_directory+'BTC_Bitfinex_day_full_14days.csv', index_col=None,dayfirst=True)

In [13]:
def check_BTC_bollinger_band_Support_Resistance(json_input,datetimeformat_string):
    BTC_Dataframe_array = [df_csv_1day,df_csv_3days,df_csv_7days,df_csv_1hour,df_csv_4hours,df_csv_6hours,df_csv_12hours]
    string_to_print1 = ""
    string_to_print2 = ""
    percent_above = 0.0
    percent_below = 0.0
    for dataframe in BTC_Dataframe_array:
        df_csv = dataframe
        #df_csv.unix_timestamp =  df_csv.time.apply(lambda t: time.mktime(datetime.datetime.strptime(str(t) ,datetimeformat_string).timetuple()))
        df_csv = df_csv.set_index(['coin', 'exchange','unix_timestamp'])
        data = list(df_csv.index.get_level_values(0).unique())
        i=0
        j=0
        for coin_name in data:
            coin_df = df_csv[df_csv.index.get_level_values(0)==coin_name] 
            #print(coin_df)
            coin_df = coin_df.reset_index()
            coin_df = coin_df.sort_values(by=['exchange','unix_timestamp']).set_index(['coin', 'exchange','unix_timestamp'])
            #print(coin_df)
            df_groupby = coin_df.groupby(['exchange'], group_keys=False)
            for key, item in df_groupby:
                req_data = df_groupby.get_group(key)
                row = req_data.tail(1)
                #print(row['timeframe'])
                if 'UPPER_BOLLINGER_BAND_VALUE' not in row:
                    continue
                BTC_UPPER_BAND,BTC_MIDDLE_BAND,BTC_LOWER_BAND = float(row['UPPER_BOLLINGER_BAND_VALUE']),float(row['MIDDLE_BOLLINGER_BAND_VALUE']),float(row['LOWER_BOLLINGER_BAND_VALUE'])
                output_from_json = json_input
                if 'price' not in output_from_json:
                    continue
                if math.isnan(BTC_UPPER_BAND):
                    continue
                current_price = float(output_from_json['price'])
                
                print(BTC_UPPER_BAND,BTC_MIDDLE_BAND,BTC_LOWER_BAND,row['timeframe'])
                difference = current_price - BTC_UPPER_BAND
                if difference >0:
                    percent_above = (difference/BTC_UPPER_BAND) * 100
                    string_to_print1 = str(current_price )+" is "+str(percent_above) +" % above case 1 "+dataframe['timeframe'] + " resistance  " + str(BTC_UPPER_BAND)
                elif difference <0:
                    percent_below = (-difference/BTC_UPPER_BAND) * 100
                    string_to_print1 = str(current_price )+" is "+str(percent_below) +" % below case 2 "+dataframe['timeframe'] + " resistance  " + str(BTC_UPPER_BAND)
                else:
                    string_to_print1 = str(current_price )+" is at case 3 "+dataframe['timeframe'] + " resistance  " + str(BTC_UPPER_BAND)
                difference = current_price - BTC_LOWER_BAND
                if difference >0:
                    percent_above = (difference/BTC_LOWER_BAND) * 100
                    string_to_print2 = str(current_price )+" is "+str(percent_above) +" % above case 4 "+dataframe['timeframe'] + " support  " + str(BTC_LOWER_BAND)
                elif difference <0:
                    percent_below = (-difference/BTC_LOWER_BAND) * 100
                    string_to_print2 = str(current_price )+" is "+str(percent_below) +" % below case 5 "+dataframe['timeframe'] + " support  " + str(BTC_LOWER_BAND)
                else:
                    string_to_print2 = str(current_price )+" is at case 6 "+dataframe['timeframe'] + " support  " + str(BTC_LOWER_BAND)
    if percent_above <= 3.0  :
        print(string_to_print1)
    if percent_below <= 3.0  :
        print(string_to_print2)

In [ ]:
websocket = WebSocket('wss://ws-feed.gdax.com')

for event in websocket:
    if event.name == "ready":
        websocket.send_json(
            type='subscribe',
            product_ids=['BTC-USD'],
            channels=['ticker']
        )
    elif event.name == "text":
        #print(event.json)
        check_BTC_bollinger_band_Support_Resistance(event.json,'%d-%m-%Y %H:%M')
        

In [ ]:
from lomond import WebSocket
import ccxt
import cryptopia
import os
import time
import talib
import logging
import requests
import datetime
import importlib
import dateutil.parser
import numpy as np
import pandas as pd
from importlib import reload
from collections import defaultdict
import sys
import inspect
import pyti
import math
from pprint import pprint
from bitmex_websocket import BitMEXWebsocket 

In [ ]:
df_csv_1day = pd.read_csv(data_directory+'BTC_Bitfinex_day_full_1day.csv', index_col=None,dayfirst=True)
df_csv_1day['timeframe'] = "1 day"
df_csv_3days = pd.read_csv(data_directory+'BTC_Bitfinex_day_full_3days.csv', index_col=None,dayfirst=True)
df_csv_3days['timeframe'] = "3 days"
df_csv_7days = pd.read_csv(data_directory+'BTC_Bitfinex_day_full_7days.csv', index_col=None,dayfirst=True)
df_csv_7days['timeframe'] = "7 days"
df_csv_1hour = pd.read_csv(data_directory+'BTC_Bitfinex_hour_full_1hour.csv', index_col=None,dayfirst=True)
df_csv_1hour['timeframe'] = "1 hour"
df_csv_4hours = pd.read_csv(data_directory+'BTC_Bitfinex_hour_full_4hours.csv', index_col=None,dayfirst=True)
df_csv_4hours['timeframe'] = "4 hours"
df_csv_6hours = pd.read_csv(data_directory+'BTC_Bitfinex_hour_full_6hours.csv', index_col=None,dayfirst=True)
df_csv_6hours['timeframe'] = "6 hours"
df_csv_12hours = pd.read_csv(data_directory+'BTC_Bitfinex_hour_full_12hours.csv', index_col=None,dayfirst=True)
df_csv_12hours['timeframe'] = "12 hours"
df_csv_14days = pd.read_csv(data_directory+'BTC_Bitfinex_day_full_14days.csv', index_col=None,dayfirst=True)

In [ ]:
def check_BTC_bollinger_band_Support_Resistance(json_input,datetimeformat_string):
    BTC_Dataframe_array = [df_csv_1day,df_csv_3days,df_csv_7days,df_csv_1hour,df_csv_4hours,df_csv_6hours,df_csv_12hours]
    string_to_print1 = ""
    string_to_print2 = ""
    percent_above = 0.0
    percent_below = 0.0
    for dataframe in BTC_Dataframe_array:
        df_csv = dataframe
        #df_csv.unix_timestamp =  df_csv.time.apply(lambda t: time.mktime(datetime.datetime.strptime(str(t) ,datetimeformat_string).timetuple()))
        df_csv = df_csv.set_index(['coin', 'exchange','unix_timestamp'])
        data = list(df_csv.index.get_level_values(0).unique())
        i=0
        j=0
        for coin_name in data:
            coin_df = df_csv[df_csv.index.get_level_values(0)==coin_name] 
            #print(coin_df)
            coin_df = coin_df.reset_index()
            coin_df = coin_df.sort_values(by=['exchange','unix_timestamp']).set_index(['coin', 'exchange','unix_timestamp'])
            #print(coin_df)
            df_groupby = coin_df.groupby(['exchange'], group_keys=False)
            for key, item in df_groupby:
                req_data = df_groupby.get_group(key)
                row = req_data.tail(1)
                #print(row['timeframe'])
                if 'UPPER_BOLLINGER_BAND_VALUE' not in row:
                    continue
                BTC_UPPER_BAND,BTC_MIDDLE_BAND,BTC_LOWER_BAND = float(row['UPPER_BOLLINGER_BAND_VALUE']),float(row['MIDDLE_BOLLINGER_BAND_VALUE']),float(row['LOWER_BOLLINGER_BAND_VALUE'])
                output_from_json = json_input
                if 'price' not in output_from_json:
                    continue
                if math.isnan(BTC_UPPER_BAND):
                    continue
                current_price = float(output_from_json['price'])
                
                #print(BTC_UPPER_BAND,BTC_MIDDLE_BAND,BTC_LOWER_BAND,row['timeframe'])
                difference = current_price - BTC_UPPER_BAND
                if difference >0:
                    percent_above = (difference/BTC_UPPER_BAND) * 100
                    string_to_print1 = str(current_price )+" is "+str(percent_above) +" % above case 1 "+dataframe['timeframe'] + " resistance  " + str(BTC_UPPER_BAND)
                elif difference <0:
                    percent_below = (-difference/BTC_UPPER_BAND) * 100
                    string_to_print1 = str(current_price )+" is "+str(percent_below) +" % below case 2 "+dataframe['timeframe'] + " resistance  " + str(BTC_UPPER_BAND)
                else:
                    string_to_print1 = str(current_price )+" is at case 3 "+dataframe['timeframe'] + " resistance  " + str(BTC_UPPER_BAND)
                difference = current_price - BTC_LOWER_BAND
                if difference >0:
                    percent_above = (difference/BTC_LOWER_BAND) * 100
                    string_to_print2 = str(current_price )+" is "+str(percent_above) +" % above case 4 "+dataframe['timeframe'] + " support  " + str(BTC_LOWER_BAND)
                elif difference <0:
                    percent_below = (-difference/BTC_LOWER_BAND) * 100
                    string_to_print2 = str(current_price )+" is "+str(percent_below) +" % below case 5 "+dataframe['timeframe'] + " support  " + str(BTC_LOWER_BAND)
                else:
                    string_to_print2 = str(current_price )+" is at case 6 "+dataframe['timeframe'] + " support  " + str(BTC_LOWER_BAND)
    if percent_above <= 3.0  :
        print(string_to_print1)
    if percent_below <= 3.0  :
        print(string_to_print2)

In [ ]:
websocket = WebSocket('wss://ws-feed.gdax.com')

for event in websocket:
    if event.name == "ready":
        websocket.send_json(
            type='subscribe',
            product_ids=['BTC-USD'],
            channels=['ticker']
        )
    elif event.name == "text":
        #print(event.json)
        check_BTC_bollinger_band_Support_Resistance(event.json,'%d-%m-%Y %H:%M')
        

In [ ]:
bitmex_exchange = ccxt.bitmex()
#print(bitmex_exchange.timeframes)
markets = bitmex_exchange.fetch_markets()
#pprint(markets)
#sys.exit("Te")
"""
if bitmex_exchange.has['fetchOHLCV']:
    for coin in markets:
        if coin['active'] == False:
            continue
        time.sleep (bitmex_exchange.rateLimit / 1000) # time.sleep wants seconds
        #print ( len(bitmex_exchange.fetch_ohlcv (coin['symbol'], '1d')))
#pprint(vars(bitmex_exchange))
"""
coins_list = set()
markets = bitmex_exchange.fetch_markets()
for row in markets:
    print(row)
    print("FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFf")
#print(coins_list)


In [14]:
ws = BitMEXWebsocket(endpoint="wss://www.bitmex.com/realtime", symbol="XBTUSD,ETHU18", api_key=None, api_secret=None)

--- request header ---
GET /realtime?subscribe=execution:XBTUSD,ETHU18,instrument:XBTUSD,ETHU18,order:XBTUSD,ETHU18,orderBookL2:XBTUSD,ETHU18,position:XBTUSD,ETHU18,quote:XBTUSD,ETHU18,trade:XBTUSD,ETHU18,margin HTTP/1.1
Upgrade: websocket
Connection: Upgrade
Host: www.bitmex.com
Origin: http://www.bitmex.com
Sec-WebSocket-Key: +CvzFQEIsctalOdVzwrmPA==
Sec-WebSocket-Version: 13


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Date: Fri, 03 Aug 2018 10:29:11 GMT
Connection: upgrade
Set-Cookie: AWSALB=/f9t6E16NaIo+FiYnL6jFY3XgzSkYx+Z8sQmIj75PHd5E45sJdQ6F5eZBOsaOl0Yy1X0Dp/V9LMqh7FHJDXlFThALbvtNQ6W3PzFh1DJ2CJb5QMwmRKjhzavGtMA; Expires=Fri, 10 Aug 2018 10:29:11 GMT; Path=/
Upgrade: websocket
Sec-WebSocket-Accept: Z0ixKEJjQ/CTA8rcVr99WZgo3kw=
Sec-WebSocket-Version: 13
WebSocket-Server: uWebSockets
Strict-Transport-Security: max-age=31536000; includeSubDomains
-----------------------


In [ ]:
ws = BitMEXWebsocket(endpoint="wss://www.bitmex.com/realtime", symbol="XBTUSD", api_key=None, api_secret=None)
df = pd.DataFrame(ws.get_ticker())
print(df)

In [15]:
ws.get_instrument()

{'askPrice': 7356.5,
 'bankruptLimitDownPrice': None,
 'bankruptLimitUpPrice': None,
 'bidPrice': 7356,
 'buyLeg': '',
 'calcInterval': None,
 'capped': False,
 'closingTimestamp': '2018-08-03T12:00:00.000Z',
 'deleverage': True,
 'expiry': None,
 'fairBasis': 0.14,
 'fairBasisRate': 0.1095,
 'fairMethod': 'FundingRate',
 'fairPrice': 7352.51,
 'front': '2016-05-13T12:00:00.000Z',
 'fundingBaseSymbol': '.XBTBON8H',
 'fundingInterval': '2000-01-01T08:00:00.000Z',
 'fundingPremiumSymbol': '.XBTUSDPI8H',
 'fundingQuoteSymbol': '.USDBON8H',
 'fundingRate': 0.0001,
 'fundingTimestamp': '2018-08-03T12:00:00.000Z',
 'hasLiquidity': True,
 'highPrice': 7595.5,
 'impactAskPrice': 7356.7277,
 'impactBidPrice': 7356.1866,
 'impactMidPrice': 7356.5,
 'indicativeFundingRate': 0.0001,
 'indicativeSettlePrice': 7352.37,
 'indicativeTaxRate': 0,
 'initMargin': 0.01,
 'insuranceFee': 0,
 'inverseLeg': '',
 'isInverse': True,
 'isQuanto': False,
 'lastChangePcnt': -0.0288,
 'lastPrice': 7356,
 'lastPric

Error : Connection is already closed.
error from callback <bound method BitMEXWebsocket.__on_error of <bitmex_websocket.BitMEXWebsocket object at 0x00000138D943CC50>>: Connection is already closed.
  File "C:\Users\Administrator\Anaconda3\lib\site-packages\websocket\_app.py", line 269, in _callback
    callback(self, *args)
  File "C:\Users\Administrator\Anaconda3\lib\site-packages\bitmex_websocket.py", line 245, in __on_error
    raise websocket.WebSocketException(error)


In [ ]:
while(ws.ws.sock.connected):
    print("Ticker: %s" % ws.get_ticker())

In [ ]:
import websocket
import _thread
import time


def on_message(ws, message):
    print(message)


def on_open(ws):
    def run(*args):
        while True:
            time.sleep(5)
            ws.send("{\"op\": \"subscribe\", \"args\": [\"orderBookL2:XBTUSD\"]}")
        time.sleep(1)
        ws.close()

    _thread.start_new_thread(run, ())


websocket.enableTrace(True)
ws = websocket.WebSocketApp("wss://www.bitmex.com/realtime",
                            on_message=on_message)
ws.on_open = on_open
ws.run_forever()